In [59]:
! rm -rf ""

In [29]:
!pip install obonet
!pip install pyvis

In [105]:
import os
import json
from typing import Dict
from collections import Counter

import random
import obonet
import pandas as pd
import numpy as np
from Bio import SeqIO

from transformers import T5Tokenizer, T5EncoderModel
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False) #.to(device)

# Load the model
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to(device);

# only GPUs support half-precision currently; if you want to run on CPU use full-precision (not recommended, much slower)
#model.full() if device=='cpu' else model.half()

In [106]:
import re
def get_embeddings(seq):
    sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", seq)))]

    ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest")

    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    # generate embeddings
    with torch.no_grad():
        embedding_repr = model(input_ids=input_ids,
                               attention_mask=attention_mask)

    # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7])
    emb_0 = embedding_repr.last_hidden_state[0]
    emb_0_per_protein = emb_0.mean(dim=0)

    return emb_0_per_protein

In [107]:
# x=get_embeddings('MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSS')
# print(len(x))
# print(x)

1024
tensor([ 0.0659, -0.0271, -0.0486,  ..., -0.0750,  0.0711,  0.0658])


In [32]:
! pip install Bio

In [34]:
predictions= [[1 for i in range(1024)]]
a=pd.DataFrame(predictions)
a.to_csv("/content/input.csv")

In [149]:
import tqdm
from Bio import SeqIO
import numpy as np
import pandas as pd
import tensorflow as tf

def predict(filepath):

    fn = filepath

    sequences = SeqIO.parse(fn, "fasta")

    ids = []
    num_sequences=sum(1 for seq in sequences)
    embeds = np.zeros((num_sequences, 1024))
    i = 0
    with open(fn, "r") as fasta_file:
    # Iterate over each sequence in the file
      for sequence in SeqIO.parse(fasta_file, "fasta"):
          # Access the sequence ID and sequence data
          seq_id = sequence.id
          seq_data = str(sequence.seq)
          embeds[i] = get_embeddings(seq_data).detach().cpu().numpy()
          print(embeds[i])
          ids.append(seq_id)
          i += 1
          if i==3:
            break #remove it for full calculation
          # print(i)

    # np.save('train_embeds.npy', embeds)
    # np.save('train_ids.npy', np.array(ids))
    # batch_shape=5120
    INPUT_SHAPE=[1024]
    num_of_labels=1500
    # from tf.keras.models import load_model
    model = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=num_of_labels, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['binary_accuracy', tf.keras.metrics.AUC()]
    )
    model.load_weights('/content/drive/MyDrive/my_model.weights.h5') #load model here
    labels_df=pd.read_csv('/content/truncated.csv')
    labels_df=labels_df.drop(columns='Unnamed: 0')
#     custom_input_tensor = np.load("/kaggle/input/t5embeds/test_embeds.npy") # add the numpy array here
    custom_input_tensor=embeds
    # custom_input_tensor=np.zeros((3, 1024))
    # print(custom_input_tensor)
    # print(len(custom_input_tensor[0]))
    # Get predictions for custom input tensor
    predictions = model.predict(custom_input_tensor)
    predictions_list=[]
    p2=[]

    cnt=0
    # 'predictions' will contain the model's output for the custom input tensor
    # print(predictions)
    for prediction in predictions:
        tmp=[]
        t2=[]
        for i in prediction:
            x=0 if i<0.4 else 1
            tmp.append(x)
            t2.append(i)
            cnt+=x
        predictions_list.append(tmp.copy())
        p2.append(t2.copy())
        print(cnt)
        cnt=0
        # break
        # print(sum(tmp))
        # print(prediction)
    # print(len(predictions))
    # print(cnt)

    # labels_df=
    # Get the column names (labels) from the original DataFrame
    label_columns = labels_df.columns

    # Convert the predictions into a DataFrame
    predictions_df = pd.DataFrame(predictions_list, columns=label_columns)
    p21=pd.DataFrame(p2, columns=label_columns)

    # Save the DataFrame to a CSV file
    predictions_df.to_csv("custom_predictions.csv", index=False) #output csv
    p21.to_csv("decimal.csv",index=False)
    return "custom_predictions.csv"


In [4]:
!pip install -q gradio

In [150]:
a=predict("/content/drive/MyDrive/abc.fasta") #/content/drive/MyDrive/abc.fasta
# print(a)

[ 0.04948842 -0.03293515  0.03247323 ... -0.04353156  0.09646282
  0.07306959]
[-0.04461637  0.064925   -0.08026287 ...  0.02672356  0.0278791
 -0.04842957]
[-0.02012802 -0.04977942  0.00789447 ... -0.03610278  0.007693
  0.10623413]


1/1 [==============================] - 0s 437ms/step
26
64
55
9
9
9
9
9
9
9
9
9
9
9
9
9
9


[[0.6468834  0.30589018 0.08692995 ... 0.78684951 0.77404817 0.7465165 ]
 [0.4408413  0.73267894 0.44538739 ... 0.15673854 0.8115624  0.46583642]
 [0.22281123 0.1046777  0.82134757 ... 0.60149203 0.47789825 0.33248386]
 ...
 [0.12337393 0.32177677 0.57687816 ... 0.85917393 0.25188528 0.63992067]
 [0.86825685 0.77709787 0.83914286 ... 0.05615919 0.33492964 0.18943627]
 [0.55108152 0.44551361 0.4567676  ... 0.98432563 0.59548267 0.42799736]]
1024
4/4 [==============================] - 0s 10ms/step
461.8934185307034


In [16]:
import gradio as gr
import pandas as pd


In [89]:
import gradio as gr
gr.Interface(
    predict,
    title = 'Protein Function Prediction using fasta file,upload a fasta file',
    inputs="file",
    outputs="file"
).launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8ee7f9fd67efae14a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1741, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1296, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r

[[0. 0. 0. ... 0. 0. 0.]]
1024
1/1 [==============================] - 0s 92ms/step
[0.4348464  0.53423107 0.42620808 ... 0.08446681 0.08856744 0.14838184]
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7866 <> https://8ee7f9fd67efae14a0.gradio.live


In [128]:
from Bio import SeqIO

# Open the FASTA file
fn = "/content/abc.fasta"
with open(fn, "r") as fasta_file:
    # Iterate over each sequence in the file
    for sequence in SeqIO.parse(fasta_file, "fasta"):
        # Access the sequence ID and sequence data
        seq_id = sequence.id
        seq_data = str(sequence.seq)

        # Process the sequence data as needed
        # print("Sequence ID:", seq_id)
        # print("Sequence Data:", seq_data)

Sequence ID: Q9CQV8
Sequence Data: MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSSWRVISSIEQKTERNEKKQQMGKEYREKIEAELQDICNDVLELLDKYLILNATQAESKVFYLKMKGDYFRYLSEVASGENKQTTVSNSQQAYQEAFEISKKEMQPTHPIRLGLALNFSVFYYEILNSPEKACSLAKTAFDEAIAELDTLNEESYKDSTLIMQLLRDNLTLWTSENQGDEGDAGEGEN
Sequence ID: Q9CQV9
Sequence Data: MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSSWRVISSIEQKTERNEKKQQMGKEYREKIEAELQDICNDVLELLDKYLILNATQAESKVFYLKMKGDYFRYLSEVASGENKQTTVSNSQQAYQEAFEISKKEMQPTHPIRLGLALNFSVFYYEILNSPEKACSLAKTAFDEAIAELDTLNEESYKDSTLIMQLLRDNLTLWTSENQGDEGDAGEGEN
